# Download a data file from the UCL server

We're using a small [example data set](http://data.cortexlab.net/singlePhase3/) recorded by [Nick Steinmetz](http://www.nicksteinmetz.com/) at UCL.

In [ ]:
import urllib.request
file_url = 'http://data.cortexlab.net/singlePhase3/data/rawDataSample.bin'
file_name = "neuropixelRaw.bin"

urllib.request.urlretrieve(file_url, file_name)

# Use of the `HSDetection` class

In [ ]:
from herdingspikes import HSDetection
from herdingspikes.probe import NeuroPixel
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# raw data location (as downloaded above)
data_path = file_name

# detection parameters
to_localize = True
cutout_start = 6
cutout_end = 30
threshold = 22
file_name = 'ProcessedSpikesNeuropixel'

In [ ]:
Probe = NeuroPixel(data_path)
H = HSDetection(Probe, to_localize, cutout_start, cutout_end, threshold,
                maa=0, maxsl=12, minsl=3, ahpthr=0, out_file_name=file_name,save_all=True)

In [ ]:
# Probe.show([0])

In [ ]:
H.DetectFromRaw()

In [ ]:
H.LoadDetected()

In [ ]:
plt.figure(figsize=(10, 10))
H.PlotTracesChannels(2000, window_size=100)

In [ ]:
plt.figure(figsize=(14, 3))
ax = plt.subplot(111)
H.PlotAll(invert=True, s=1, alpha=0.2, ax=ax)
ax.set_xlim((0,500))

# Use of the `HSClustering` class

In [ ]:
from hs2 import HSClustering
# Load from file
#C = Clustering(['results/ProcessedSpikes_mea1k.bin', 'results/ProcessedSpikes_mea1k.bin'], cutout_length=41)

# Or if the spikes are already in memory
# simply load from the Detection class
C = HSClustering(H)

In [ ]:
%%time
C.ShapePCA(pca_ncomponents=2, pca_whiten=True);

In [ ]:
%%time
C.CombinedClustering(alpha=5, bandwidth=5., bin_seeding=False, n_jobs=-1,cluster_subset=50000)

In [ ]:
plt.figure(figsize=(14, 4))
ax = plt.subplot(111)
C.PlotAll(invert=True, s=1, ax=ax, max_show=100000,show_labels=False)
plt.title("MeanShift, bandwidth=.3, no bin seeding");
ax.set_xlim((0,500))


In [ ]:
units = range(16)
C.PlotShapes(units)

In [ ]:
C.PlotNeighbourhood(1,radius=2, alpha=0.8);

In [ ]:
# save results:
C.SaveHDF5(file_name+"_sorted.hdf5")